### EDA (Exploratory Data Analysis) & Data Cleaning

In [ ]:

##import das bibliotecas e adequando colunas, linhas e formato de números

from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd
import pandas_gbq as gbq
from google.oauth2 import service_account
from google.cloud.bigquery_storage import BigQueryReadClient
import seaborn as sns
import numpy as np
import os
import re
import json

# Carrega o .env
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente
if os.name == "nt":  # Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# Parâmetros injetados pelo Papermill ou definidos manualmente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.humanresources_employee"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# Configurar o cliente do BigQuery com project e location dinâmicos
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


# Verificar se a configuração está correta
print("Credenciais do BigQuery:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

# Verifica se a variável está configurada
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))




In [ ]:
print("Tabelas a processar:", tables_to_process)


In [ ]:
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project="desafioadventureworks-446600", location="us-central1")


# # Configurar o cliente do BigQuery
# client = bigquery.Client()

# Nome do dataset e tabela
dataset_id = 'raw_data'


In [ ]:
# Configurar Pandas para exibir todas as colunas e todas as linhas completas
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Listar tabelas no dataset
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)




In [ ]:
# Inicializar o cliente do BigQuery
client = bigquery.Client(credentials=credentials, project="desafioadventureworks-446600", location="us-central1")

# Configurar o cliente do BigQuery com project e location dinâmicos
credentials = service_account.Credentials.from_service_account_file(credentials_path)

# Inicializar o cliente do BigQuery Storage
bqstorage_client = BigQueryReadClient(credentials=credentials)


# Dicionário para armazenar DataFrames processados
processed_data = {}

# Processar tabelas e armazenar DataFrames
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela
    table_name = input_table.split(".")[-1]  # Extrai o nome da tabela
    
    # Etapa 1: Ler os dados da tabela do BigQuery com pyarrow
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    EDA_humanresources_employee_raw = client.query(query).to_dataframe(bqstorage_client=bqstorage_client)

    # Etapa 2: Transformar JSON em formato tabular
    print("Transformando os dados para formato tabular...")

    # Verificar se há colunas com dados em formato JSON
    if EDA_humanresources_employee_raw.shape[1] == 1 and isinstance(EDA_humanresources_employee_raw.iloc[0, 0], str):
        try:
            print("Normalizando dados JSON...")
            # Substituir `null` por `None` e carregar o JSON
            EDA_humanresources_employee = pd.json_normalize(
                EDA_humanresources_employee_raw.iloc[:, 0].apply(lambda x: json.loads(x.replace("null", "None")))
            )
        except Exception as e:
            print(f"Erro ao normalizar JSON: {e}")
            EDA_humanresources_employee = EDA_humanresources_employee  # Caso falhe, mantém os dados brutos
    else:
        EDA_humanresources_employee = EDA_humanresources_employee_raw

    # Armazenar o DataFrame limpo em um dicionário
    processed_data[table_name] = EDA_humanresources_employee
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Após o loop, exibir uma mensagem de conclusão
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
print(EDA_humanresources_employee_raw.iloc[:, 0].head())


In [ ]:
# def clean_and_load_json(value):
#     """Função para corrigir e carregar JSON."""
#     try:
#         # Substituir `null` por `None` e carregar o JSON
#         value = value.replace("null", "null")
#         return json.loads(value)
#     except Exception as e:
#         print(f"Erro ao processar JSON: {e}, valor problemático: {value}")
#         return None  # Retorna None se o valor for inválido

# # Normalizar os dados JSON
# print("Normalizando os dados JSON...")
# try:
#     EDA_humanresources_employee = pd.json_normalize(
#         EDA_humanresources_employee_raw.iloc[:, 0].apply(clean_and_load_json)
#     )
#     print("Dados normalizados com sucesso!")
# except Exception as e:
#     print(f"Erro ao normalizar os dados JSON: {e}")
#     EDA_humanresources_employee = EDA_humanresources_employee_raw  # Mantém os dados originais em caso de erro

# print(f"Tabela processada: {input_table}")
# print(EDA_humanresources_employee.head())




def clean_and_load_json(value):
    """Função para corrigir e carregar JSON."""
    try:
        # Substituir `null` por `None` e carregar o JSON
        value = value.replace("null", "None")
        return json.loads(value)
    except Exception as e:
        print(f"Erro ao processar JSON: {e}, valor problemático: {value}")
        return None  # Retorna None se o valor for inválido

# Normalizar os dados JSON
print("Normalizando os dados JSON...")
try:
    EDA_humanresources_employee = pd.json_normalize(
        EDA_humanresources_employee_raw.iloc[:, 0].apply(clean_and_load_json)
    )
    print("Dados normalizados com sucesso!")

    # Atribuir tipos às colunas
    EDA_humanresources_employee['businessentityid'] = EDA_humanresources_employee['businessentityid'].astype('int64', errors='ignore')
    EDA_humanresources_employee['nationalidnumber'] = EDA_humanresources_employee['nationalidnumber'].astype('int64', errors='ignore')
    EDA_humanresources_employee['loginid'] = EDA_humanresources_employee['loginid'].astype('int64', errors='ignore')
    EDA_humanresources_employee['jobtitle'] = EDA_humanresources_employee['jobtitle'].astype('str', errors='ignore')
    EDA_humanresources_employee['birthdate'] = pd.to_datetime(EDA_humanresources_employee['birthdate'], errors='coerce')
    EDA_humanresources_employee['maritalstatus'] = EDA_humanresources_employee['maritalstatus'].astype('str', errors='ignore')
    EDA_humanresources_employee['gender'] = EDA_humanresources_employee['gender'].astype('str', errors='ignore')
    EDA_humanresources_employee['hiredate'] = pd.to_datetime(EDA_humanresources_employee['hiredate'], errors='coerce')
    EDA_humanresources_employee['salariedflag'] = EDA_humanresources_employee['salariedflag'].astype('bool')
    EDA_humanresources_employee['vacationhours'] = EDA_humanresources_employee['vacationhours'].astype('int64', errors='ignore')
    EDA_humanresources_employee['sickleavehours'] = EDA_humanresources_employee['sickleavehours'].astype('int64', errors='ignore')
    EDA_humanresources_employee['currentflag'] = EDA_humanresources_employee['currentflag'].astype('bool')
    EDA_humanresources_employee['rowguid'] = EDA_humanresources_employee['rowguid'].astype('str', errors='ignore')
    EDA_humanresources_employee['modifieddate'] = pd.to_datetime(EDA_humanresources_employee['modifieddate'], errors='coerce')
    EDA_humanresources_employee['organizationnode'] = EDA_humanresources_employee['organizationnode'].astype('str', errors='ignore')

    print("Tipos atribuídos com sucesso!")
except Exception as e:
    print(f"Erro ao normalizar os dados JSON: {e}")
    EDA_humanresources_employee = EDA_humanresources_employee_raw  # Mantém os dados originais em caso de erro

print(f"Tabela processada: {input_table}")

print(EDA_humanresources_employee.info())

In [ ]:
#dimensões do df antes de remover duplicatas

EDA_humanresources_employee.shape

In [ ]:
print("Colunas disponíveis no DataFrame limpo (cleaned):", EDA_humanresources_employee.columns)

# Identificar duplicatas com base em 'businessentityid'
duplicatas = EDA_humanresources_employee[
    EDA_humanresources_employee.duplicated(subset=['businessentityid'], keep=False)
]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas.sort_values(by=['businessentityid', 'modifieddate'])

    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")


In [ ]:
# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(EDA_humanresources_employee)}")

#cópia dados brutos
raw_data_bkp_2_sem_duplicatas = EDA_humanresources_employee.copy()


In [ ]:
# Ordenar o DataFrame por 'businessentityid' e 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.sort_values(by=['businessentityid', 'modifieddate'])

print(EDA_humanresources_employee)




In [ ]:
#Certifique-se de que as colunas de datas está sendo reconhecida corretamente como contendo valores nulos (NaN em pandas). (Não pode object)

print(EDA_humanresources_employee.info())


In [ ]:
# Identificar as colunas de data
date_columns = ['birthdate', 'hiredate', 'modifieddate']

# Converter todas as colunas para datetime
for col in date_columns:
    EDA_humanresources_employee[col] = pd.to_datetime(
        EDA_humanresources_employee[col], errors='coerce'
    )

# Criar uma cópia do DataFrame para exportação no formato JSON
datas_formatadas = EDA_humanresources_employee.copy()

# Formatar colunas no formato ISO 8601 para BigQuery e tratar nulos como null
for col in date_columns:
    datas_formatadas[col] = datas_formatadas[col].apply(
        lambda x: x.isoformat() if pd.notnull(x) else None  # Certifique-se de que é datetime
    )

print(EDA_humanresources_employee.info())


In [ ]:
# Iterar por todas as colunas do DataFrame

for column in EDA_humanresources_employee.columns:
    # Verificar valores ausentes na coluna
    missing_rows = EDA_humanresources_employee[EDA_humanresources_employee[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
    # Mostrar as primeiras linhas ausentes (limitar para não poluir a saída)
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")



In [ ]:
# Preencher 'modifieddate' ausente ou igual a 'hiredate', pois pode ser a ultima data de modificação no sistema.
EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'].isnull() | (EDA_humanresources_employee['modifieddate'] == pd.Timestamp('1900-01-01')), 'modifieddate'] = EDA_humanresources_employee['hiredate']

# Exibir as linhas ajustadas
print("Linhas onde 'modifieddate' foi ajustado para 'hiredate':")
print(EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'] == EDA_humanresources_employee['hiredate']])


In [ ]:
# Criar uma cópia do DataFrame para exportação no formato JSON
ajustes_date_time = EDA_humanresources_employee.copy()

In [ ]:
# valores únicos por coluna

valores_unicos = EDA_humanresources_employee.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

In [ ]:
# dropar colunas vazias

In [ ]:
# Padronizar textos em title ou upper
EDA_humanresources_employee['jobtitle'] = EDA_humanresources_employee['jobtitle'].str.strip().str.title()
EDA_humanresources_employee['gender'] = EDA_humanresources_employee['gender'].str.strip().str.upper()
EDA_humanresources_employee['maritalstatus'] = EDA_humanresources_employee['maritalstatus'].str.strip().str.upper()


# Verificar valores únicos para garantir a padronização
print("Valores únicos em 'jobtitle':", EDA_humanresources_employee['jobtitle'].unique())
print("Valores únicos em 'gender':", EDA_humanresources_employee['gender'].unique())
print("Valores únicos em 'gender':", EDA_humanresources_employee['maritalstatus'].unique())


In [ ]:
# Identificar colunas numéricas para análise 
numeric_columns = ['sickleavehours', 'vacationhours']

# Exibir estatísticas descritivas
print(EDA_humanresources_employee[numeric_columns].describe())

# Calcular limites para outliers (IQR - Intervalo Interquartil)
for col in numeric_columns:
    q1 = EDA_humanresources_employee[col].quantile(0.25)
    q3 = EDA_humanresources_employee[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Filtrar outliers
    outliers = EDA_humanresources_employee[(EDA_humanresources_employee[col] < lower_bound) | (EDA_humanresources_employee[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


In [ ]:
# Definir regex para validar números (exemplo: apenas dígitos, 9 caracteres)
# acrescentei para ver se tinha um padrão, mas não tem
regex = r'^\d{9}$'

# Verificar valores inválidos
invalid_nationalid = EDA_humanresources_employee[~EDA_humanresources_employee['nationalidnumber'].astype(str).str.match(regex)]
print(f"Valores inválidos em 'nationalidnumber':\n{invalid_nationalid['nationalidnumber']}")


In [ ]:
# Criar um backup do DataFrame tratado
EDA_humanresources_employee_bkp_v2 = EDA_humanresources_employee.copy()

# Verificar o tamanho do backup e as primeiras linhas
print(f"Backup criado com {len(EDA_humanresources_employee_bkp_v2)} linhas.")
print(EDA_humanresources_employee_bkp_v2.head())


In [ ]:
# Verificar e documentar colunas existentes
print("Colunas mantidas no dataset:", EDA_humanresources_employee.columns.tolist())


In [ ]:
# Listar colunas binárias esperadas
binary_columns = ['currentflag', 'salariedflag']

# Verificar valores únicos em colunas binárias
for col in binary_columns:
    unique_values = EDA_humanresources_employee[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}") 



In [ ]:
# Contar valores em 'currentflag' e 'salariedflag'
print("Distribuição de 'currentflag':")
print(EDA_humanresources_employee['currentflag'].value_counts())

print("\nDistribuição de 'salariedflag':")
print(EDA_humanresources_employee['salariedflag'].value_counts())


#se vale a pena deletar ou não a coluna currentflag, já que só tem 1 valor e é true ?!

In [ ]:
# 1. Verificar se todos os funcionários ativos têm currentflag = True, pois deveria ser false = demitido/desligado
print("Funcionários ativos errados:", EDA_humanresources_employee[EDA_humanresources_employee['currentflag'] != True])

# 2. Validar datas
print("Contratações futuras:", EDA_humanresources_employee[EDA_humanresources_employee['hiredate'] > pd.Timestamp.now()])
print("Modifieddate antes de hiredate:", EDA_humanresources_employee[EDA_humanresources_employee['modifieddate'] < EDA_humanresources_employee['hiredate']])





In [ ]:
# Ajustar o formato das colunas de data para atender ao BigQuery
EDA_humanresources_employee['modifieddate'] = pd.to_datetime(EDA_humanresources_employee['modifieddate'], errors='coerce').dt.date
EDA_humanresources_employee['birthdate'] = pd.to_datetime(EDA_humanresources_employee['birthdate'], errors='coerce').dt.date
EDA_humanresources_employee['hiredate'] = pd.to_datetime(EDA_humanresources_employee['hiredate'], errors='coerce').dt.date

# Atualizar o dicionário processed_data com o DataFrame ajustado
processed_data['humanresources_employee'] = EDA_humanresources_employee

# # Exportar tabelas para o BigQuery no formato CSV
# for table_name, df_cleaned in processed_data.items():
#     output_table = f"{output_dataset}.{table_name}"
    
#     print(f"Exportando tabela {table_name} para o BigQuery...")

#     # Definir o esquema explicitamente
#     schema = [
#         bigquery.SchemaField("birthdate", "DATE"),
#         bigquery.SchemaField("businessentityid", "INTEGER"),
#         bigquery.SchemaField("currentflag", "BOOLEAN"),
#         bigquery.SchemaField("gender", "STRING"),
#         bigquery.SchemaField("hiredate", "DATE"),
#         bigquery.SchemaField("jobtitle", "STRING"),
#         bigquery.SchemaField("loginid", "STRING"),
#         bigquery.SchemaField("maritalstatus", "STRING"),
#         bigquery.SchemaField("modifieddate", "DATE"),
#         bigquery.SchemaField("nationalidnumber", "INTEGER"),
#         bigquery.SchemaField("organizationnode", "STRING"),
#         bigquery.SchemaField("rowguid", "STRING"),
#         bigquery.SchemaField("salariedflag", "BOOLEAN"),
#         bigquery.SchemaField("sickleavehours", "INTEGER"),
#         bigquery.SchemaField("vacationhours", "INTEGER"),
#     ]

#     job_config = bigquery.LoadJobConfig(
#         source_format=bigquery.SourceFormat.CSV,
#         skip_leading_rows=0,
#         write_disposition="WRITE_TRUNCATE",
#         schema=schema,  # Especifica os tipos de dados explicitamente
#     )

#     # Exportar para o BigQuery
#     job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
#     job.result()

#     print(f"Tabela {table_name} exportada com sucesso para {output_table}.")


In [185]:
# Função para ajustar o prefixo do nome da tabela
def adjust_table_name(table_name):
    # Garante que o nome da tabela não tenha prefixo duplicado
    if table_name.startswith("stg_stg_"):
        table_name = table_name.replace("stg_stg_", "stg_")
    return table_name if table_name.startswith(("fact_", "dim_", "stg_")) else f"stg_{table_name}"

# Remover duplicatas e ajustar os nomes no dicionário
processed_data = {
    adjust_table_name(table_name): df
    for table_name, df in processed_data.items()
}

# Garantir que apenas tabelas únicas sejam exportadas
unique_processed_data = {k: v for k, v in processed_data.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_processed_data.items():
    output_table = f"{output_dataset}.{table_name}"
    schema = [
        bigquery.SchemaField("birthdate", "DATE"),
        bigquery.SchemaField("businessentityid", "INTEGER"),
        bigquery.SchemaField("currentflag", "BOOLEAN"),
        bigquery.SchemaField("gender", "STRING"),
        bigquery.SchemaField("hiredate", "DATE"),
        bigquery.SchemaField("jobtitle", "STRING"),
        bigquery.SchemaField("loginid", "STRING"),
        bigquery.SchemaField("maritalstatus", "STRING"),
        bigquery.SchemaField("modifieddate", "DATE"),
        bigquery.SchemaField("nationalidnumber", "INTEGER"),
        bigquery.SchemaField("organizationnode", "STRING"),
        bigquery.SchemaField("rowguid", "STRING"),
        bigquery.SchemaField("salariedflag", "BOOLEAN"),
        bigquery.SchemaField("sickleavehours", "INTEGER"),
        bigquery.SchemaField("vacationhours", "INTEGER"),
    ]
    filtered_schema = [field for field in schema if field.name in df_cleaned.columns]

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=0,
        write_disposition="WRITE_TRUNCATE",
        schema=filtered_schema,
    )
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")


Tabela stg_humanresources_employee exportada com sucesso para desafioadventureworks-446600.raw_data_cleaned.stg_humanresources_employee.


## ESTATÍSTICA DESCRITIVA

In [ ]:
# Selecionar colunas relevantes para análise descritiva
cols_para_analise = ['sickleavehours', 'vacationhours']

# Garantir que as datas estejam no formato correto
EDA_humanresources_employee['hire_year'] = pd.to_datetime(EDA_humanresources_employee['hiredate']).dt.year

# Adicionar a nova coluna à lista
cols_para_analise.append('hire_year')

# Gerar estatísticas descritivas
analise_descritiva = EDA_humanresources_employee[cols_para_analise].describe(include='all')

# Substituir NaN em colunas numéricas por 0, e em outras colunas por '-'
for col in cols_para_analise:
    if analise_descritiva[col].dtype.kind in 'ifc':  # Tipos numéricos
        analise_descritiva[col] = analise_descritiva[col].fillna(0)
    else:
        analise_descritiva[col] = analise_descritiva[col].fillna('-')

# Gerar estatísticas descritivas
resultado_descritivo = analise_descritiva.describe(include='all')

print(analise_descritiva)
